In [1]:
import pandas as pd

# steps
# 1. find out how accurate forecast is -> metric to see how acurate it is
# 2. connect weather in counties to weather stations /latitude+longitude
# 3. add installed capacity, electricity & gas prices to train
# 4. model distance from station as individual customer forecast probability -> correlation of usage and weather


#Metric 
# mean absolute error

In [4]:
client = pd.read_csv("../data/client.csv")
# all clients of company: product type, county, isbusiness, datablockid (same time for forecast) SAME as train
# per date 
# new info: eic_count (business type) + installed capacity

ePrices = pd.read_csv("../data/electricity_prices.csv")
# forecasted prices for a specific date
# has origin date + data_block_id
# forecast per hour

gasPrices = pd.read_csv("../data/gas_prices.csv")
# forecast per date
# lowest & highest price -> difference
# data block id

weatherPred = pd.read_csv("../data/forecast_weather.csv")
# longitude & latitude -> connect to county!!
# hours ahead
# temp, dewpoint, cloudcover high + low + total + mid (total=high+low), 10m east(u) wind, 10m north(v) wind, direct solar radiation, surface solar (direct + diffuse), snowfall, total precipitation
# data block, 
# origin date in DATE valid for 48 hours!!!
# forecast date in hours -> hours ahead + origin date

weatherHist = pd.read_csv("../data/historical_weather.csv")
# datetime per HOUR
# latitude longitude
# temp, dewpoint, rain, snowfall, surface pressure, cloudcover low, mid, high, total, shortwave + diffuse radiation, direct solar,
# data block it

train = pd.read_csv("../data/train.csv")
# data per hour
# county, isbusiness, product type, data block id
# row_id == index -> unimportant
# prediction unit = unique identifier for county, business, product type  == client id (new ones can appear) -> get eic count + capacity over similar indicators
# target
# is consumption (0,1)!! -> makes target + or -

In [47]:
weatherHist

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.4,12.0,0.0,0.0,1015.8,4,4,0,0,6.694444,3,0.0,0.0,0.0,57.6,21.7,1
1,2021-09-01 00:00:00,14.4,12.0,0.0,0.0,1015.8,4,4,0,0,6.694444,3,0.0,0.0,0.0,57.6,21.7,1
2,2021-09-01 00:00:00,14.0,12.0,0.0,0.0,1010.6,7,8,0,0,4.944444,353,0.0,0.0,0.0,57.6,22.2,1
3,2021-09-01 00:00:00,14.4,12.8,0.0,0.0,1014.9,6,7,0,0,5.833333,348,0.0,0.0,0.0,57.6,22.7,1
4,2021-09-01 00:00:00,14.4,12.8,0.0,0.0,1014.9,6,7,0,0,5.833333,348,0.0,0.0,0.0,57.6,22.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710795,2023-05-30 10:00:00,10.8,4.1,0.4,0.0,1018.7,40,0,67,0,1.333333,283,440.0,264.0,176.0,59.7,26.2,637
1710796,2023-05-30 10:00:00,11.4,2.9,0.3,0.0,1018.8,41,0,69,0,1.527778,302,453.0,283.0,170.0,59.7,26.7,637
1710797,2023-05-30 10:00:00,9.5,3.2,0.1,0.0,1018.9,47,0,79,0,3.416667,302,477.0,312.0,165.0,59.7,27.2,637
1710798,2023-05-30 10:00:00,11.0,1.6,0.0,0.0,1018.6,31,0,52,0,3.472222,309,535.0,400.0,135.0,59.7,27.7,637
